In [3]:
import pandas as pd
import openpyxl

data = pd.read_excel(r"C:\Users\GARVITA\OneDrive\Desktop\prostate cancer\datasheetformetastatic_RGCIRC.xlsx", engine="openpyxl")


In [2]:
data.columns

Index(['Year of diagnosis', 'Age at diagnosis', 'Height', 'Weight', 'BMI',
       'Family history', 'Histology', 'Charlson Comorbidity Index', 'T', 'N',
       'M', 'Bones', 'Nodes', 'Viscera', 'PSA levels', 'Gleason Score', 'ECOG',
       'Nadir PSA', 'PSA doubling time< 6 months', 'BP',
       'Survival after 3 years'],
      dtype='object')

In [3]:
data= data.drop('Height', axis=1)
data= data.drop('Weight', axis=1)
data= data.drop('BMI', axis=1)
data.dropna(how='all', inplace=True)

In [4]:
# Replace missing values with the median of the column
data['Family history'] = data['Family history'].fillna("No")
data['BP'] = data['BP'].fillna("No")
data['Survival after 3 years'] = data['Survival after 3 years'].fillna("No")
data['Bones'] = data['Bones'].fillna("No")
data['Nodes'] = data['Nodes'].fillna("No")
data['Viscera'] = data['Viscera'].fillna("No")
data['Gleason Score']=data['Gleason Score'].fillna(0)
data['Nadir PSA']=data['Nadir PSA'].fillna(0)
data['PSA doubling time< 6 months']= data['PSA doubling time< 6 months'].fillna("No")
data['ECOG']=data['ECOG'].fillna(data['ECOG'].mean())
data['PSA levels']=data['PSA levels'].fillna(data['PSA levels'].mean())

In [5]:
dummies = pd.get_dummies(data[['Family history', 'Histology', 'Bones', 'Nodes', 'Viscera','M', 'N',
                               'PSA doubling time< 6 months', 'BP', 'Survival after 3 years']])
data = pd.concat([data, dummies], axis=1)
data= data.drop(['Family history', 'Histology', 'N', 'M', 'Bones', 'Nodes',
       'Viscera','PSA doubling time< 6 months', 'BP', 'Survival after 3 years',
       'Family history_No',
       'Histology_Acinar AdencoarciNoma',
       'Histology_Acinar Adenocarcinoma ',
       'Histology_Adenocarcinoma ',
       'Histology_acinar Adenocarcinoma', 'M_M1', 'Bones_No', 'Nodes_No', 'Viscera_No',
       'PSA doubling time< 6 months_No','BP_No',
       'Survival after 3 years_No', 'Survival after 3 years_No  '], axis=1)

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import pandas as pd
X = data.drop(['Survival after 3 years_Yes'], axis=1)
y = data['Survival after 3 years_Yes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
print(data.isna().sum())

Year of diagnosis                            0
Age at diagnosis                             0
Charlson Comorbidity Index                   0
T                                            0
PSA levels                                   0
Gleason Score                                0
ECOG                                         0
Nadir PSA                                    0
Family history_Yes                           0
Histology_Acinar Adenocarcinoma              0
Histology_Adenocarcinoma                     0
Histology_Neuroendocrine Carcinoma           0
Histology_Rhabdomyosarcoma                   0
Histology_Signet Ring Cell Adenocarcinoma    0
Bones_Yes                                    0
Nodes_Yes                                    0
Viscera_Yes                                  0
M_M1b                                        0
N_N0                                         0
N_N1                                         0
N_N2                                         0
PSA doubling 

In [8]:
# OVERSAMPLING using smote
from imblearn.over_sampling import SMOTE
smt=SMOTE()
X_train, y_train=smt.fit_resample(X_train, y_train)
#X_train_sm, y_train_sm=smt.fit_resample(X_train, y_train)

In [9]:
from tensorflow.keras import layers, models, optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")])





In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test), batch_size=64, verbose=1)
history

Epoch 1/1000
7/7 [==============================] - 2s 107ms/step - loss: 0.8431 - accuracy: 0.7488 - val_loss: 0.8936 - val_accuracy: 0.7073
Epoch 2/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.4227 - accuracy: 0.7799 - val_loss: 0.8572 - val_accuracy: 0.6585
Epoch 3/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.3774 - accuracy: 0.7967 - val_loss: 0.8842 - val_accuracy: 0.7683
Epoch 4/1000
7/7 [==============================] - 0s 16ms/step - loss: 0.3636 - accuracy: 0.8325 - val_loss: 0.9478 - val_accuracy: 0.7439
Epoch 5/1000
7/7 [==============================] - 0s 16ms/step - loss: 0.2912 - accuracy: 0.8660 - val_loss: 1.0733 - val_accuracy: 0.7439
Epoch 6/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2748 - accuracy: 0.8876 - val_loss: 1.1586 - val_accuracy: 0.8049
Epoch 7/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2660 - accuracy: 0.8900 - val_loss: 1.2825 - val_accuracy: 0.8049
Epoch 8/1000

Epoch 59/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2063 - accuracy: 0.9043 - val_loss: 1.4325 - val_accuracy: 0.8293
Epoch 60/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2176 - accuracy: 0.9019 - val_loss: 1.3490 - val_accuracy: 0.7439
Epoch 61/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2365 - accuracy: 0.8876 - val_loss: 1.4306 - val_accuracy: 0.7561
Epoch 62/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.2292 - accuracy: 0.8995 - val_loss: 1.5682 - val_accuracy: 0.7317
Epoch 63/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.2468 - accuracy: 0.8900 - val_loss: 1.3462 - val_accuracy: 0.8049
Epoch 64/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2591 - accuracy: 0.8732 - val_loss: 1.4733 - val_accuracy: 0.7317
Epoch 65/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2611 - accuracy: 0.8756 - val_loss: 1.6462 - val_accuracy: 0.6707
Epoch 

7/7 [==============================] - 0s 17ms/step - loss: 0.2434 - accuracy: 0.8876 - val_loss: 0.7207 - val_accuracy: 0.7927
Epoch 174/1000
7/7 [==============================] - 0s 16ms/step - loss: 0.2568 - accuracy: 0.8684 - val_loss: 0.7132 - val_accuracy: 0.7805
Epoch 175/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2288 - accuracy: 0.8923 - val_loss: 0.6205 - val_accuracy: 0.8171
Epoch 176/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2716 - accuracy: 0.8660 - val_loss: 0.8694 - val_accuracy: 0.7317
Epoch 177/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.3133 - accuracy: 0.8684 - val_loss: 0.5953 - val_accuracy: 0.7683
Epoch 178/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.2699 - accuracy: 0.8708 - val_loss: 0.6180 - val_accuracy: 0.8171
Epoch 179/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2456 - accuracy: 0.8852 - val_loss: 0.5975 - val_accuracy: 0.7805
Epoch 180/1000

7/7 [==============================] - 0s 17ms/step - loss: 0.1897 - accuracy: 0.9043 - val_loss: 0.6584 - val_accuracy: 0.7683
Epoch 288/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1822 - accuracy: 0.9163 - val_loss: 0.8161 - val_accuracy: 0.7317
Epoch 289/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1684 - accuracy: 0.9211 - val_loss: 0.8577 - val_accuracy: 0.7805
Epoch 290/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2253 - accuracy: 0.8947 - val_loss: 0.7799 - val_accuracy: 0.7317
Epoch 291/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.3502 - accuracy: 0.8541 - val_loss: 0.7479 - val_accuracy: 0.6951
Epoch 292/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.3298 - accuracy: 0.8445 - val_loss: 0.5994 - val_accuracy: 0.7561
Epoch 293/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2903 - accuracy: 0.8589 - val_loss: 0.6738 - val_accuracy: 0.7561
Epoch 294/1000

7/7 [==============================] - 0s 15ms/step - loss: 0.2175 - accuracy: 0.8900 - val_loss: 0.7888 - val_accuracy: 0.8049
Epoch 402/1000
7/7 [==============================] - 0s 16ms/step - loss: 0.2793 - accuracy: 0.8828 - val_loss: 0.9875 - val_accuracy: 0.6707
Epoch 403/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.2824 - accuracy: 0.8756 - val_loss: 0.5622 - val_accuracy: 0.7927
Epoch 404/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.2839 - accuracy: 0.8708 - val_loss: 0.7033 - val_accuracy: 0.6951
Epoch 405/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.3045 - accuracy: 0.8636 - val_loss: 0.5721 - val_accuracy: 0.7927
Epoch 406/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2492 - accuracy: 0.8995 - val_loss: 0.6729 - val_accuracy: 0.7561
Epoch 407/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.2654 - accuracy: 0.8684 - val_loss: 0.7615 - val_accuracy: 0.7561
Epoch 408/1000

7/7 [==============================] - 0s 16ms/step - loss: 0.2432 - accuracy: 0.9091 - val_loss: 0.8575 - val_accuracy: 0.7561
Epoch 516/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2039 - accuracy: 0.9115 - val_loss: 0.6371 - val_accuracy: 0.7805
Epoch 517/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1930 - accuracy: 0.9163 - val_loss: 0.6379 - val_accuracy: 0.8293
Epoch 518/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1723 - accuracy: 0.9163 - val_loss: 0.7894 - val_accuracy: 0.7927
Epoch 519/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1703 - accuracy: 0.9139 - val_loss: 0.7459 - val_accuracy: 0.7927
Epoch 520/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1864 - accuracy: 0.9091 - val_loss: 0.6272 - val_accuracy: 0.7683
Epoch 521/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2275 - accuracy: 0.8756 - val_loss: 0.6564 - val_accuracy: 0.7927
Epoch 522/1000

7/7 [==============================] - 0s 16ms/step - loss: 0.2541 - accuracy: 0.8852 - val_loss: 0.8516 - val_accuracy: 0.7439
Epoch 630/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2368 - accuracy: 0.8947 - val_loss: 0.6353 - val_accuracy: 0.8049
Epoch 631/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1903 - accuracy: 0.9115 - val_loss: 0.6198 - val_accuracy: 0.7927
Epoch 632/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1909 - accuracy: 0.9163 - val_loss: 0.5864 - val_accuracy: 0.8171
Epoch 633/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1954 - accuracy: 0.9019 - val_loss: 0.6261 - val_accuracy: 0.8171
Epoch 634/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1853 - accuracy: 0.9163 - val_loss: 0.6314 - val_accuracy: 0.8049
Epoch 635/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1861 - accuracy: 0.9043 - val_loss: 0.6927 - val_accuracy: 0.8293
Epoch 636/1000

7/7 [==============================] - 0s 16ms/step - loss: 0.1578 - accuracy: 0.9211 - val_loss: 1.0385 - val_accuracy: 0.7561
Epoch 744/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1556 - accuracy: 0.9234 - val_loss: 0.7233 - val_accuracy: 0.8049
Epoch 745/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1745 - accuracy: 0.9258 - val_loss: 0.8416 - val_accuracy: 0.7439
Epoch 746/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1509 - accuracy: 0.9450 - val_loss: 1.0178 - val_accuracy: 0.7561
Epoch 747/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1386 - accuracy: 0.9330 - val_loss: 0.7287 - val_accuracy: 0.8049
Epoch 748/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1320 - accuracy: 0.9378 - val_loss: 0.8595 - val_accuracy: 0.7561
Epoch 749/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1560 - accuracy: 0.9306 - val_loss: 0.7550 - val_accuracy: 0.7927
Epoch 750/1000

7/7 [==============================] - 0s 17ms/step - loss: 0.1396 - accuracy: 0.9330 - val_loss: 1.0562 - val_accuracy: 0.7561
Epoch 858/1000
7/7 [==============================] - 0s 16ms/step - loss: 0.1392 - accuracy: 0.9306 - val_loss: 0.8335 - val_accuracy: 0.7927
Epoch 859/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1320 - accuracy: 0.9402 - val_loss: 0.8110 - val_accuracy: 0.8415
Epoch 860/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2002 - accuracy: 0.9043 - val_loss: 0.8204 - val_accuracy: 0.8415
Epoch 861/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2493 - accuracy: 0.8923 - val_loss: 1.1045 - val_accuracy: 0.7805
Epoch 862/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2278 - accuracy: 0.9019 - val_loss: 1.0239 - val_accuracy: 0.8049
Epoch 863/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.2240 - accuracy: 0.8995 - val_loss: 1.1270 - val_accuracy: 0.7683
Epoch 864/1000

7/7 [==============================] - 0s 15ms/step - loss: 0.1328 - accuracy: 0.9426 - val_loss: 1.0298 - val_accuracy: 0.7439
Epoch 972/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1236 - accuracy: 0.9450 - val_loss: 0.9996 - val_accuracy: 0.7927
Epoch 973/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1266 - accuracy: 0.9330 - val_loss: 0.9987 - val_accuracy: 0.7927
Epoch 974/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1162 - accuracy: 0.9522 - val_loss: 0.9390 - val_accuracy: 0.8049
Epoch 975/1000
7/7 [==============================] - 0s 17ms/step - loss: 0.1298 - accuracy: 0.9426 - val_loss: 1.0918 - val_accuracy: 0.7683
Epoch 976/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1263 - accuracy: 0.9498 - val_loss: 1.1519 - val_accuracy: 0.7683
Epoch 977/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.1197 - accuracy: 0.9450 - val_loss: 1.0936 - val_accuracy: 0.8171
Epoch 978/1000

In [14]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Confusion Matrix:', confusion_matrix(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))



3/3 [==============================] - 0s 4ms/step
Accuracy: 0.8414634146341463
Confusion Matrix: [[39  7]
 [ 6 30]]
Precision: 0.8108108108108109
Recall: 0.8333333333333334
F1 Score: 0.8219178082191781
ROC-AUC Score: 0.8405797101449275


In [12]:
# Extract the loss values from the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create a figure and axis object
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the training and validation loss over time
ax.plot(train_loss, label='Training Loss')
ax.plot(val_loss, label='Validation Loss')

# Set axis labels, title, legend, and other formatting options
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.set_title('Model Loss Over Time')
ax.legend(loc='best')

# Display the plot
plt.show()


NameError: name 'plt' is not defined

In [ ]:
# Extract the accuracy values from the training history
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Create a figure and axis object
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the training and validation loss over time
ax.plot(train_acc, label='Training Accuracy')
ax.plot(val_acc, label='Validation Accuracy')

# Set axis labels, title, legend, and other formatting options
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
ax.set_title('Model Accuracy Over Time')
ax.legend(loc='best')

# Display the plot
plt.show()